In [1]:
import cv2
import numpy as np
import zipfile
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization , Dropout
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
!pip install face_recognition
import face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566164 sha256=13d8395cdf0d25274a1ce05400d13aec107868d884384c1834f916f2186ef741
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [2]:
zip_file = zipfile.ZipFile('datasetimages3.zip')

In [6]:
zip_file.extractall('datasetimages3')

In [7]:
train_data_dir = '/content/datasetimages2/datasetimages3/train'
test_data_dir = '/content/datasetimages2/datasetimages3/test'

In [8]:
# Preprocess training images without ImageDataGenerator
train_images = []
train_labels = []

for filename in os.listdir(train_data_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        img_path = os.path.join(train_data_dir, filename)
        img = cv2.imread(img_path)

        # Preprocess image: resize, normalization, etc.
        img = cv2.resize(img, (224, 224))  # Example: resize to 224x224
        img = img / 255.0  # Normalize pixel values to range [0, 1]

        train_images.append(img)

        # Extract label from filename (assuming a naming convention)
        label = filename.split('_')[0]  # Example: Assuming labels are in filename
        train_labels.append(label)

# Convert lists to NumPy arrays
train_images = np.array(train_images)
train_labels = np.array(train_labels)

In [9]:
test_images = []
test_labels = []

for filename in os.listdir(train_data_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        img_path = os.path.join(train_data_dir, filename)
        img = cv2.imread(img_path)

         # Extract label from filename (assuming a naming convention)
        label = filename.split('_')[0]  # Example: Assuming labels are in filename
        test_labels.append(label)

# Convert lists to NumPy arrays
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [11]:
# Create the image data generator
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load the training and testing data
training_set = train_datagen.flow_from_directory('/content/datasetimages3/datasetimages3/train',
                                                 target_size=(64, 64), batch_size=32, class_mode='categorical')
test_set = test_datagen.flow_from_directory(test_data_dir, target_size=(64, 64), batch_size=32, class_mode='categorical')

# Create the model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(5, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x=training_set, epochs=3, validation_data='/content/datasetimages3/datasetimages3/test')

# Evaluate the model
loss, accuracy = model.evaluate(x=test_set, verbose=0)
print('Loss: {}'.format(loss))
print('Accuracy: {}'.format(accuracy))

# Save the model
model.save('image_classifier.h5')

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: Must provide at least one structure